## 最小二乘法 (Least Square)

最小二乘法问题是求解向量 $x$ 满足 $\min \Vert Ax - b\Vert_2$, 先只考虑实数域.

其中 $A$ 是 $m\times n$ 的实矩阵, $b$ 为 $m\times 1$ 的实向量.

我们只考虑 $m \geqslant n$ 的情形. 否则若 $m < n$, 方程 $Ax - b = 0$ 很可能有解. 实际运用中, 一般 $m\gg n$. 

考虑范数的平方

$$f(x) = (Ax - b)^T(Ax - b) = x^TA^TAx - 2x^TA^Tb + b^Tb.$$

是二次型. 其梯度

$$\nabla f(x) = 2A^TAx - 2A^Tb.$$

极值在 $A^TAx = A^Tb$ 时取到, 该式被称为法方程.

<br>

**Theorem 1** $A^TAx = A^Tb$ 必有解: 若记 $A^\dag$ 为 $A$ 的 Moore-Penrose 伪逆, 则 $x = A^\dag b$ 是一个解.

证: 若 $A$ 为零矩阵则平凡, 否则其秩 $r>0$. 

设 $A$ 的奇异值分解为 $A = U_rDV_r^T$, 其中 $U_r,D,V_r^T$ 的形状为 $m\times r, r\times r, r\times n$. 
则 
$$A^TAA^\dag b = V_rDU_r^TU_rDV_r^TV_rD^{-1}U_r^Tb = V_rDU_r^Tb = A^Tb.$$

<br>

**Theorem 2** $\Vert Ax - b\Vert_2$ 最小当且仅当 $A^TAx = A^Tb$.

证: 由余弦定理, $\forall x,y\in \mathbb R^m$,

$$\Vert Ay - b\Vert_2^2 = \Vert Ax - b\Vert_2^2 + \Vert A(y-x)\Vert_2^2 - 2(y-x)^TA^T(Ax - b).$$

特别地, 取 $x$ 满足 $A^TAx = A^Tb$, 则上式变成

$$\Vert Ay - b\Vert_2^2 = \Vert Ax - b\Vert_2^2 + \Vert A(y-x)\Vert_2^2 \geqslant \Vert Ax - b\Vert_2^2.$$

说明此时 $\Vert Ax - b\Vert_2$ 必然是最小值, 取等当且仅当 $A(y-x) = 0$, 从而
 $$Ay = Ax \Rightarrow A^TAy = A^TAx = A^Tb.$$

 
<br>

**Theorem 3** $x = A^\dag b$ 是所有最小二乘解中, 二范数最小的 (之一).

证: 令 $x = A^\dag b$, 若 $A^TAy = A^Tb = A^TAx$, 则 $A^TA(y-x)=0$.

$$\Vert y\Vert_2^2 = \Vert x\Vert_2^2 +\Vert y-x\Vert_2^2 + 2x^T(y-x)\geqslant
\Vert x\Vert_2^2 + 2x^T(y-x) $$

设 $A$ 的奇异值分解为 $A = U_rDV_r^T$, 其中 $U_r,D,V_r^T$ 的形状为 $m\times r, r\times r, r\times n$. 
则 

$$0=A^TA(y-x) = V_rD^2V_r^T(y-x).$$

从而 $$V_r^T(y-x) = D^{-2}V_r^T \left(V_rD^2V_r^T(y-x)\right) = 0.$$

于是 $x^T(y-x) = b^TU_rD^{-1}V_r^T(y-x) = 0$, 证毕.

### Cholesky方法

现在假定 $A$ 列满秩, 那么 $A^TA$ 是 $n\times n$ 的可逆矩阵. 实际上, 若 $m\gg n$ 且数据随机, 那么 $A$ 满列秩的概率为 $1$ .

$A^TA$ 是正定的对称矩阵, 其 Cholesky 分解是一种易于计算的 LU 分解,  $A^TA = R^TR$, 其中 $R$ 是上三角矩阵.

则需要解决方程组 $A^TAx = A^Tb$ 变成 $R^TRx = A^Tb$. 可以通过求解两次三角方程组解得 $x$.

In [48]:
import numpy as np
import math

# 解可逆三角方程组 Ax = b, 设置 upper = 1表示上三角, upper = 0表示下三角
def SolveTriangular(A,b,upper = 1):
    n = A.shape[0]
    x = np.zeros(n,dtype=A.dtype)
    if upper : # 上三角
        for k in range(n-1,-1,-1):
            x[k] = (b[k] - np.dot(A[k,k+1:],x[k+1:])) / A[k,k]
    else: # 下三角
        for k in range(n):
            x[k] = (b[k] - np.dot(A[k,:k],x[:k])) / A[k,k]
    return x

# Cholesky 分解实半正定对称矩阵, 返回上三角
def Cholesky(A):
    R = np.zeros(A.shape,dtype=A.dtype)
    n = A.shape[0]
    for k in range(n):
        R[k,k] = math.sqrt(max(0, (A[k,k] - np.dot(R[:k,k].conj(),R[:k,k])).real))
        R[k,k+1:] = (A[k,k+1:] - R[:k,k].conj() @ R[:k,k+1:]) / R[k,k]
    return R

# 求解最小二乘问题 Ax = b
def LeastSquareCholesky(A,b):
    # R^TRx = A^Tb
    R = Cholesky(A.T.conj() @ A)
    return SolveTriangular(R , SolveTriangular(R.T.conj(), A.T.conj() @ b, 0))

# 求解一个例子并与 NumPy 自带的函数比较, 进行验证
m , n = 10000 , 80
np.random.seed(0)
A = np.random.random((m,n)) - 0.5
b = np.random.rand(m)
print('Minimum            = ',np.linalg.norm(A @ LeastSquareCholesky(A,b) - b))
print('Minimum (by NumPy) = ',np.linalg.lstsq(A,b,rcond=None)[1][0]**0.5)

Minimum            =  57.44406333454826
Minimum (by NumPy) =  57.444063334548254


### QR方法

虽然可以通过求解方程组 $A^TAx = A^Tb$ 得到最小二乘问题的解, 但该方法囿于矩阵 $A^TA$ 的条件数是 $A$ 的平方, 数值稳定性差.
   
一个常用的解决最小二乘问题的算法是利用 QR 分解, 回到原始形态的问题 $\Vert Ax - b \Vert_2$:

假定 $A$ 有 QR 分解 $A = Q\left[\begin{matrix}R \\ 0\end{matrix}\right]$, 其中 $Q$ 为 $m\times m$ 的正交矩阵, $R$ 为 $n\times n$ 的上三角矩阵. 
$$\Vert Ax - b \Vert_2 = \Vert Q^TAx - Q^Tb \Vert_2 = \Vert \left[\begin{matrix}R \\ 0\end{matrix}\right]x - Q^Tb \Vert_2.$$

若记 $Q^Tb = \left[\begin{matrix}c_1 \\ c_2\end{matrix}\right]$, 那么

$$\Vert \left[\begin{matrix}R \\ 0\end{matrix}\right]x - Q^Tb \Vert_2 = 
\Vert \left[\begin{matrix}Rx-c_1 \\ -c_2\end{matrix}\right] \Vert_2\geqslant \Vert c_2\Vert_2.$$

若 $A$ 满列秩, 则 $R$ 满秩, 取等当且仅当 $Rx= c_1$, 解该三角方程组即可. 

### QR方法

实际上, 若 $A$ 满列秩, QR 方法是化简版本的 Cholesky 方法. 这是因为 $A = QR$, 且 $Q^TQ = I_n$, 那么

$$A^TAx = A^Tb\ \Rightarrow \  R^TQ^TQRx = R^TQ^Tb\  \Rightarrow \ Rx = Q^Tb.$$

从上式也可以看出, $Q$ 不需要是 $m\times m$ 的, 即使是用 Gram-Schmidt 得到的 $m\times n$ 的也可以, 这在 $m\gg n$ 的情况下节省了时空复杂度.

In [38]:
# 集成的 Gram-Schmidt
# 参数 modified = 0 or 1, 表示是否是 MGS
# 参数 reortho = 0 or 1, 表示是否重正交化
# 只返回正交阵, 可以通过 Q^TA 计算 R
def GramSchmidtQR(A,modified = 1,reortho = 1):
    reortho += 1
    Q = A.copy() 
    n = A.shape[1]
    R = np.zeros((n,n),dtype=A.dtype)
    for k in range(n):
        if modified:
            for _ in range(reortho):
                for i in range(k):
                    Q[:,k] -= np.dot(Q[:,i].conj(),Q[:,k]) * Q[:,i]
        else:
            for _ in range(reortho):          
                Q[:,k] -= Q[:,:k] @ (Q[:,:k].T.conj() @ Q[:,k])
        Q[:,k] /= np.linalg.norm(Q[:,k])
        R[k+1:,k] = 0
    return Q 

# 求解最小二乘问题 Ax = b
# 参数 modified = 0 or 1, 表示是否是 MGS
# 参数 reortho = 0 or 1, 表示是否重正交化
def LeastSquareGramSchmidt(A,b,modified = 1,reortho = 1):
    Q = GramSchmidtQR(A,modified,reortho)
    return SolveTriangular(Q.T.conj() @ A, Q.T.conj() @ b)


# 求解一个例子并与 NumPy 自带的函数比较, 进行验证
m , n = 10000 , 80
np.random.seed(0)
A = np.random.random((m,n)) - 0.5
b = np.random.rand(m)
print('Minimum            = ',np.linalg.norm(A @ LeastSquareCholesky(A,b) - b))
print('Minimum (by NumPy) = ',np.linalg.lstsq(A,b,rcond=None)[1][0]**0.5)

Minimum            =  57.44406333454826
Minimum (by NumPy) =  57.444063334548254


### 正则化 (Regularization) / 岭回归 (Ridge Regression)

给定 $A\in \mathbb R^{m\times n}, b\in \mathbb R^m$, $\lambda \in \mathbb R^*$, 求解 
$\min \{\Vert Ax - b\Vert_2^2 + \lambda \Vert x\Vert_2^2\}$. 这要求权衡 $x$ 与 $Ax - b$, 都不能太大.

一个方案是求导:

$$\nabla = 2A^TAx - 2A^Tb + 2\lambda x.$$

另一个方案是正则化问题可以转化为普通的最小二乘问题:

$$\Vert Ax - b\Vert_2^2 + \lambda \Vert x\Vert_2^2=
\Vert \left[\begin{matrix}A \\ \sqrt \lambda I_n\end{matrix}\right]x - \left[\begin{matrix}b \\ 0\end{matrix}\right]
\Vert_2^2.$$

### 复数域

假若是复数域上的最小二乘问题: $\min \Vert Ax - b\Vert_2$, 其中 $A\in \mathbb C^{m\times n}$, $b\in \mathbb C^m$, $x\in \mathbb C^n$. 

虽然不能求导, 但显然**Theorem 2**等及其证明可以推广到复数域, 将所有转置换成转置共轭即可:

**Theorem 1'** $A^*Ax = A^*b$ 必有解: 若记 $A^\dag$ 为 $A$ 的 Moore-Penrose 伪逆, 则 $x = A^\dag b$ 是一个解.
<br>

**Theorem 2'** $\Vert Ax - b\Vert_2$ 最小当且仅当 $A^*Ax = A^*b$.

**Theorem 3'** $x = A^\dag b$ 是所以最小二乘解中, 二范数最小的 (之一).

且 Cholesky 方法与 QR 方法都是可以相同推广的.

In [53]:
# 求解一个例子并与 NumPy 自带的函数比较, 进行验证
m , n = 10000 , 80
np.random.seed(1)
A = np.random.random((m,n)) - 0.5 + np.random.random((m,n))*1j - 0.5j
b = np.random.rand(m) + np.random.rand(m)*1j
print('Minimum by Chole   = ',np.linalg.norm(A @ LeastSquareCholesky(A,b) - b))
print('Minimum by GS      = ',np.linalg.norm(A @ LeastSquareGramSchmidt(A,b) - b))
print('Minimum (by NumPy) = ',np.linalg.lstsq(A,b,rcond=None)[1][0]**0.5)

Minimum by Chole   =  81.37931526811875
Minimum by GS      =  81.37931526811875
Minimum (by NumPy) =  81.37931526811866


### 空间约束

在约束条件 $Cx = d$ 下求解最小二乘问题 $\min \Vert Ax-b\Vert_2$. 其中 $C\in \mathbb R^{p\times n}$, $p\leqslant n$ 且 $C$ 满行秩.

先处理约束条件, 假设 $C$ 有 LQ 分解 $C = \left[\begin{matrix}L & 0 \end{matrix}\right] \left[\begin{matrix}Q_1^*\\ Q_2^*\end{matrix}\right] = LQ_1^*$, 若令 $y_1 = Q_1^*x$, 及 $y_2 = Q_2^*x$, 则

$$d = Cx = LQ_1^*x = Ly_1.$$

而 

$$Ax - b = A\left[\begin{matrix}Q_1 & Q_2\end{matrix}\right]\left[\begin{matrix}Q_1^*\\ Q_2^*\end{matrix}\right]x-b
= AQ_1y_1 + AQ_2y_2 - b.
$$

由于 $y_1 = L^{-1}d$, 上式可以重新整理为

$$AQ_1y_1 + AQ_2y_2 - b = AQ_2y_2 - (b - AQ_1L^{-1}d).$$

于是问题变成求解 $y_2$ 使得 $\Vert AQ_2y_2 - (b - AQ_1L^{-1}d)\Vert_2$ 最小, 这是普通的最小二乘问题. 

解出 $y_2$ 以及 $y_1 = L^{-1}d$ 后注意

$$\left[\begin{matrix}Q_1^*\\ Q_2^*\end{matrix}\right] x = \left[\begin{matrix}y_1\\ y_2\end{matrix}\right]$$

从而

$$x = \left[\begin{matrix}Q_1 & Q_2\end{matrix}\right]\left[\begin{matrix}y_1\\ y_2\end{matrix}\right]=Q_1y_1+Q_2y_2
=Q_1L^{-1}d + Q_2y_2$$


### 增广矩阵

最小二乘问题 $\min \Vert Ax - b\Vert_2$ 的解等价于如下 Hermite 线性方程组的解
$$\left[\begin{matrix}I & A\\A^* & 0\end{matrix}\right]
\left[\begin{matrix}r \\ x\end{matrix}\right]=
\left[\begin{matrix}b \\ 0\end{matrix}\right].$$

证: 必要性: 该线性方程组的解满足 $r + Ax = b$ 且 $A^*r = 0\Rightarrow A^*Ax = A^*(b-r) = A^*b$.

充分性: 若 $A^*Ax = A^*b$, 取 $r = b-Ax$ 则必有 $A^*r = 0$.

<br>

对于有空间约束 $Cx = d$ 的最小二乘问题, 其解等价于如下 Hermite 线性方程组的解
$$\left[\begin{matrix}I & A & 0\\A^* & 0 & C^* \\ 0 & C & 0\end{matrix}\right]
\left[\begin{matrix}r \\ x \\ z\end{matrix}\right]=
\left[\begin{matrix}b \\ 0 \\ d\end{matrix}\right].$$

证: 采取前一节的记号, 假设 $C$ 有 LQ 分解 $C = \left[\begin{matrix}L & 0 \end{matrix}\right] \left[\begin{matrix}Q_1^*\\ Q_2^*\end{matrix}\right]$, 那么最终问题等价于求解
$$\min \Vert AQ_2y_2 - (b - AQ_1L^{-1}d)\Vert_2$$
因此可写成线性方程组 $r + AQ_2y_2 = b - AQ_1L^{-1}d$ 且 $Q_2^*A^*r = 0$.

注意 $x=Q_1L^{-1}d + Q_2y_2$, 上式即等价于 $r + Ax = b$ 且 $Q_2^*A^*r = 0$.

又因为 $Q_2^*Q_1=0$, 即 $Q_1$的列空间 是 $Q_2^*$ 的正交补, 说明 $A^*r$ 在 $Q_1$ 的列空间中. 因为 $L$ 可逆, 这等价于在 $C^*=Q_1^*L^*$ 的列空间中, 亦可写作: 存在 $z$, 使得 $ A^*r + C^*z = 0$.

### 拉格朗日乘数法

在约束条件 $Cx = d$ 下求解最小二乘问题 $\min \Vert Ax-b\Vert_2$ 数学上也可以用拉格朗日乘数法, 即考虑一个待定系数 $\lambda \in \mathbb R^p$.

$$f(x,\lambda) = \Vert Ax-b\Vert_2^2 - \lambda^T (Cx - d).$$

则

$$\nabla = 2A^TAx - 2A^Tb - C^T\lambda .$$

这说明极值点需要同时满足 $Cx = d$ 且 $A^TAx - A^Tb$ 在 $C^T$ 的列空间中. 这与上一节的增广线性方程组一致.
